<a href="https://colab.research.google.com/github/Paul-mwaura/Natural-Language-Processing/blob/main/IkoKaziKE_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:#800000"> Fitting a Word2Vec Model on the IkoKaziKE data</span>

<span style="color:orange">Here,I call in the data,check for missing values,keep what I need and use the BOW model </span>

In [ ]:
#Import the necessary libraries
!pip install gensim
import re
import string 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import nltk
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from bs4 import BeautifulSoup

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#import the dataset
df_tweet=pd.read_csv("twitter-job-hunter-chatbot.csv")
df_tweet.head()

,Datetime,Text,Source,harsh tag,Favourite Count,Retweets,6,7,submitter_name
0,18/07/2020 11:46,"b""Ladies!!! #IkoKaziKE #IkoKazi Today, I reall...",b'Mimimkenya7',"['IkoKaziKE', 'IkoKazi']",NaN,775.0,NaN,NaN,Brian Cheye
1,18/07/2020 11:18,b'#IkoKaziKe #IkoKazi Looking for a Audit Trai...,b'MtandaoPromoter',"['IkoKaziKe', 'IkoKazi']",NaN,5.0,NaN,NaN,Brian Cheye
2,18/07/2020 09:08,b'@moneychapKE are an online crowdfunding and ...,b'KameneAndJalas',"['KameneAndJalas', 'ikoJob', 'TwendeKaziKe', '...",NaN,15675.0,NaN,NaN,Brian Cheye
3,18/07/2020 09:07,b'We are an online crowdfunding and fundraisin...,b'moneychapKE',"['IkoKazi', 'IkoKaziKE']",NaN,79.0,NaN,NaN,Brian Cheye
4,18/07/2020 08:46,b'Striding into the weekend with a productive ...,b'amunsoft',"['webdeveloper', 'business', 'IkoKazi', 'IkoKa...",NaN,2.0,NaN,NaN,Brian Cheye


In [ ]:
print(df_tweet.shape)

(17222, 9)


In [ ]:
#df_tweet.info()

In [ ]:
#Variable Data types
df_tweet.dtypes

Datetime            object
Text                object
Source              object
harsh tag           object
Favourite Count    float64
Retweets           float64
6                  float64
7                  float64
submitter_name      object
dtype: object

# Cleaning the Data

In [ ]:
df_tweet.isnull().sum()

Datetime               0
Text                   0
Source                 5
harsh tag          16690
Favourite Count      533
Retweets               1
6                   4048
7                   4048
submitter_name         1
dtype: int64

Since harsh tag has very many missing values that will be unreasonable to drop,
We fill the missing values with "no tag" the drop the rest of the na's

In [ ]:
df_tweet["harsh tag"].fillna("no tag",inplace=True)

<span style="color:green">We can now drop the na values without worrying of dropping so many observations </span>

In [ ]:
df_tweet.dropna(inplace=True)

In [ ]:
#confirm in there are still na values
df_tweet.isnull().sum()

Datetime           0
Text               0
Source             0
harsh tag          0
Favourite Count    0
Retweets           0
6                  0
7                  0
submitter_name     0
dtype: int64

In [ ]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13174 entries, 4047 to 17220
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Datetime         13174 non-null  object 
 1   Text             13174 non-null  object 
 2   Source           13174 non-null  object 
 3   harsh tag        13174 non-null  object 
 4   Favourite Count  13174 non-null  float64
 5   Retweets         13174 non-null  float64
 6   6                13174 non-null  float64
 7   7                13174 non-null  float64
 8   submitter_name   13174 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.0+ MB


In [ ]:
#check if we have any dulicates
print(df_tweet.duplicated().any())

True


In [ ]:
#drop the duplicates
df_tweet.drop_duplicates(keep=False,inplace=True)

In [ ]:
#Confirm if the duplicates have been dropped
print(df_tweet.duplicated().any())

False


In [ ]:
df_tweet.describe()

,Favourite Count,Retweets,6,7
count,13170.000000,13170.000000,1.317000e+04,13170.000000
mean,3.668185,4.948595,6.157548e+02,254.979879
std,37.654959,81.840446,1.355763e+04,1588.867862
min,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000e+00,0.000000
50%,1.000000,0.000000,0.000000e+00,0.000000
75%,2.000000,2.000000,0.000000e+00,0.000000
max,2722.000000,4167.000000,1.496395e+06,28407.000000


# <u><span style="color:orange"> Dealing with outliers </span></u>

In [ ]:
from scipy import stats
num = ['Retweets']

for i, col in enumerate(num):
    z=np.abs(stats.zscore(df_tweet[col]))
print(z)

[0.06046867 0.06046867 0.04824931 ... 0.06046867 0.04824931 0.02381059]


In [ ]:
#subset the data with observations whose z score is less than 3
df1=df_tweet[(z<3)]
print (f"Data frame with outliers had : {(df_tweet.shape[0])}")
print(f"Data frame without outliers has : {df1.shape[0]}")


Data frame with outliers had : 13170
Data frame without outliers has : 13149


In [ ]:
#remove spaces and convert the variable names to lower
df1.columns=df1.columns.str.lower().str.strip()

In [ ]:
#Cleaning text,lower remove punctuation,white space
#create a function then use it to clean the column texts
'''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
def clean_text(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]','',text)   # question marks,square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    return text           


In [ ]:
df1['text']= df1['text'].apply(lambda x:clean_text(x))
df1['source']=df1['source'].apply(lambda x:clean_text(x))

In [ ]:
df1.head()

,datetime,text,source,harsh tag,favourite count,retweets,6,7,submitter_name
4047,2020-07-18 10:16:18+00:00,job seekers free cover letter template ikok...,simoningari,no tag,1.0,0.0,4342.0,4994.0,Dennis Mwaniki
4048,2020-07-18 10:09:08+00:00,get the latest and best car shade and all cust...,simonmagak,no tag,0.0,0.0,28.0,92.0,Dennis Mwaniki
4049,2020-07-18 10:05:52+00:00,join jobalertke for latest updates on ikok...,simoningari,no tag,2.0,1.0,4342.0,4994.0,Dennis Mwaniki
4050,2020-07-18 10:05:22+00:00,hey people im looking for a farm manager with ...,vidolebaridi,no tag,0.0,0.0,141.0,485.0,Dennis Mwaniki
4051,2020-07-18 10:05:16+00:00,job hunting can be emotionally draining to en...,simoningari,no tag,0.0,0.0,4342.0,4994.0,Dennis Mwaniki


In [ ]:
#use a subset of what I need.
df_w2v = df1[["datetime" ,"text","source","harsh tag","retweets"]]  #get rid of 6,7 and submitter_name

In [ ]:
df_w2v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13149 entries, 4047 to 17220
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   13149 non-null  object 
 1   text       13149 non-null  object 
 2   source     13149 non-null  object 
 3   harsh tag  13149 non-null  object 
 4   retweets   13149 non-null  float64
dtypes: float64(1), object(4)
memory usage: 616.4+ KB


In [ ]:
df_w2v.dtypes

datetime      object
text          object
source        object
harsh tag     object
retweets     float64
dtype: object

# <span style="color:maroon">Word2Vec Model </span>

Here we will implement the Word2vec model using the Gensim library
Procedure 
<br> 0.Create/Identify your corpus </br>
<br> 1.Tokenize </br>   //preprocessing
<br> 2.Remove Stopwords</br>
<br> 4.Modelling with BOW
<br> 0. Split Test and Train Data <br>

In [ ]:
#1.Tokenizing

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Creating the model

In [ ]:
def prep_text(text) :

     # 1. Remove HTML.
    my_text = BeautifulSoup(text,"html.parser").get_text()
    
    #Tokenize words
    tokens=nltk.word_tokenize(my_text)
    
    #Define stopwords
    stops=set(stopwords.words("english"))
    # Convert the stopwords list to "set" data type.
    #stops = set(nltk.corpus.stopwords.words("english"))
    
    #Remove the stopwords
    words=[x for x in tokens if  not x in stops]
   
    
    #Remove the stopwords
    #for i in range(0,len(words)):
       # words[i]=[x for x in words if  not x in stops]
    
    return words

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Performing the iterative preps on each of the statements
df_w2v['preped_text']=df_w2v['text'].apply(prep_text)

In [ ]:
df_w2v.head(10)

,datetime,text,source,harsh tag,retweets,preped_text
4047,2020-07-18 10:16:18+00:00,job seekers free cover letter template ikok...,simoningari,no tag,0.0,"[job, seekers, free, cover, letter, template, ..."
4048,2020-07-18 10:09:08+00:00,get the latest and best car shade and all cust...,simonmagak,no tag,0.0,"[get, latest, best, car, shade, customised, ca..."
4049,2020-07-18 10:05:52+00:00,join jobalertke for latest updates on ikok...,simoningari,no tag,1.0,"[join, jobalertke, latest, updates, ikokazike,..."
4050,2020-07-18 10:05:22+00:00,hey people im looking for a farm manager with ...,vidolebaridi,no tag,0.0,"[hey, people, im, looking, farm, manager, spec..."
4051,2020-07-18 10:05:16+00:00,job hunting can be emotionally draining to en...,simoningari,no tag,0.0,"[job, hunting, emotionally, draining, ensure, ..."
4052,2020-07-18 09:59:06+00:00,job whatsapp groups to join ikokazike,simoningari,no tag,0.0,"[job, whatsapp, groups, join, ikokazike]"
4053,2020-07-18 09:47:12+00:00,i am looking for good carpenter to do some wor...,babananii,no tag,1.0,"[looking, good, carpenter, work, recommendatio..."
4054,2020-07-18 09:46:00+00:00,kot were still taking applications for regiona...,qsskenya,no tag,1.0,"[kot, still, taking, applications, regional, h..."
4055,2020-07-18 09:40:05+00:00,the east african community is looking for a h...,findjobskenya,no tag,0.0,"[east, african, community, looking, human, res..."
4056,2020-07-18 09:39:26+00:00,vacancy at miritini medical labtech qualificat...,hossanamiritini,no tag,3.0,"[vacancy, miritini, medical, labtech, qualific..."


In [ ]:

from gensim.models import Word2Vec

In [ ]:
word2vec=Word2Vec(df_w2v['preped_text'],min_count=2)      # Specifies to include only words in the model that appear atleast 10 times in the corpus

In [ ]:
corpus = vocabulary

from pprint import pprint
for i in corpus:
  pprint(i)
  

Streaming output truncated to the last 5000 lines.
'truck'
'bono'
'tiktok'
'ebooks'
'absa'
'didnt'
'properties'
'€'
'orinda'
'thetiredhusband'
'rottentradeunion'
'safaricomtwins'
'undergraduate'
'reasons'
'renewable'
'irena'
'pa'
'distance'
'narratives'
'avatar'
'sxxl'
'jacob'
'juma'
'crowd'
'kamiti'
'jumwa'
'applytoday'
'hiringalert'
'statement'
'filing'
'return'
'seeks'
'mugs'
'pretty'
'fits'
'perfectly'
'hipesaapp'
'stockholm'
'nil'
'library'
'salescoordinator'
'lower'
'primary'
'cum'
'hts'
'aspiring'
'grey'
'practices'
'swedish'
'moves'
'represent'
'disabilities'
'culinary'
'teachingjobs'
'potato'
'sqm'
'thick'
'easier'
'sail'
'groupd'
'healthcarejobs'
'keys'
'beyond'
'selflearning'
'techniques'
'collections'
'trainers'
'trans'
'waitress'
'elearning'
'mediator'
'administrators'
'humanresourcesjobs'
'reproductive'
'midlevel'
'stream'
'fortinet'
'kasee'
'eyesonjusticemakau'
'ford'
'wrong'
'okay'
'stuff'
'tired'
'quit'
'oversight'
'scenes'
'sides'
'tangatangathugs'
'cgiar'
'iitacgiar'

In [ ]:
#Lets view the vocabulary
vocabulary= word2vec.wv.vocab
print(vocabulary)

{'job': <gensim.models.keyedvectors.Vocab object at 0x7f4a12d67ba8>, 'seekers': <gensim.models.keyedvectors.Vocab object at 0x7f4a12d67c18>, 'free': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9668>, 'cover': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a96a0>, 'letter': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a96d8>, 'template': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9710>, 'ikokazike': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9748>, 'patakazike': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9780>, 'gethired': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a97b8>, 'jobseeker': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a97f0>, 'get': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9828>, 'latest': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9860>, 'best': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a9898>, 'car': <gensim.models.keyedvectors.Vocab object at 0x7f49ec7a98d0>, 'sha

In [ ]:
# preping text for BOW :remove stopwords and lemmatize

def counting(text) :
    my_text = BeautifulSoup(text,"html.parser").get_text()
    
    #Tokenize words
    tokens=nltk.word_tokenize(my_text)
    
    #Define stopwords
    stops=set(stopwords.words("english"))
    
    
    #Remove the stopwords
    words=[x for x in tokens if  not x in stops]
    #Confirming with the frequencies
    wordscount={}
    """
    for data in my_text:"""
    for word in words:
        if word not in wordscount.keys():
            wordscount[word]= 1
        else:
            wordscount[word] += 1
    #print(wordscount)
    
    return wordscount

        

In [ ]:
#Performing the iterative preps on each of the statements
df_w2v['count_text']=df_w2v['text'].apply(counting)

In [ ]:

print(df_w2v['count_text'])

4047     {'job': 1, 'seekers': 1, 'free': 1, 'cover': 1...
4048     {'get': 1, 'latest': 1, 'best': 1, 'car': 1, '...
4049     {'join': 1, 'jobalertke': 1, 'latest': 1, 'upd...
4050     {'hey': 1, 'people': 1, 'im': 1, 'looking': 1,...
4051     {'job': 3, 'hunting': 2, 'emotionally': 1, 'dr...
                               ...                        
17216    {'buildbackabetterke': 1, '’': 1, 'entrepreneu...
17217    {'found': 1, 'love': 1, 'would': 1, 'like': 1,...
17218    {'buildbackabetterke': 1, 'failure': 1, 'oppor...
17219    {'usikose': 1, 'friday': 1, 'bonus': 2, 'depos...
17220    {'fedora': 1, 'hats': 1, 'order': 1, 'contact'...
Name: count_text, Length: 13149, dtype: object


# Model Analysis

Lets now explore what we created

Finding Vectors for a Word
We know that the Word2Vec model converts words to their corresponding vectors. Let's see how we can view vector representation of any particular word.

In [ ]:
processed_docs = df_w2v['preped_text']


In [ ]:
vec1=word2vec.wv['ikokazike']
print(vec1)

[-5.54232955e-01 -4.81889099e-01  5.91237433e-02  3.03024918e-01
  9.44689885e-02  9.27233398e-01  1.83373952e-04  1.05516481e+00
 -7.83046007e-01  6.29244924e-01  8.82131934e-01 -7.44483590e-01
  1.31684557e-01  1.60425872e-01  1.94904640e-01 -6.79205835e-01
  1.67316663e+00 -7.45778441e-01 -1.49767369e-01  1.52189255e-01
  3.64285409e-02  6.53382957e-01  8.05194318e-01  5.83402276e-01
  5.84888279e-01  4.50510494e-02 -4.00441855e-01 -1.89938188e-01
 -4.64086644e-02 -5.26790559e-01  6.78501070e-01  2.98360825e-01
 -2.14223132e-01  4.59468812e-01 -6.44974470e-01 -4.85641837e-01
 -8.06011558e-01 -6.14962399e-01 -8.59094322e-01 -5.46414554e-01
  4.95353013e-01 -1.14516258e-01 -7.82403171e-01  2.37181112e-01
 -2.08670989e-01  3.96496058e-01 -4.17221397e-01 -2.47163504e-01
  4.37374592e-01 -9.12282705e-01  5.29400051e-01  5.83507836e-01
 -2.22437754e-01  5.58966875e-01 -9.23356637e-02  2.67006159e-01
 -1.00000501e+00 -1.43269986e-01 -7.39824712e-01 -5.92684150e-01
 -4.38090920e-01  1.09140

Finding Similar Words

Earlier we said that contextual information of the words is not lost using Word2Vec approach. We can verify this by finding all the words similar to the word "seeking".

In [ ]:
similar=word2vec.wv.most_similar('job')
print(similar)

[('twilio', 0.9698964357376099), ('remotework', 0.9638205170631409), ('rose', 0.9631476402282715), ('developer', 0.962658166885376), ('head', 0.961931586265564), ('engineer', 0.9618531465530396), ('pooling', 0.9588603973388672), ('bss', 0.9588249921798706), ('doximity', 0.9584932327270508), ('curriculum', 0.958372950553894)]


In [ ]:
import gensim
import pickle

pickle.dump(word2vec, open("word2vec.pkl", 'wb'))
dictionary.save("dict.gensim")

In [ ]:


lda = gensim.models.ldamodel.LdaModel.load("dict.gensim")